<a href="https://colab.research.google.com/github/Jimin980921/DeepLearning/blob/master/MNIST_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 불러오기

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True) 

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

tensorflow는 계산을 위해 고효율의 c++백엔드를 사용함. 
tensorflow는 session을 사용함.

tensorflow는 먼저 그래프를 구성하고, 그래프를 세션을 통해 실행함.


tensorflow코드를보다 유연하게 작성할 수 있게 해주는 InteractiveSession클래스를 사용함.
이 클래스는 계산 그래프와 실행작업을 분리시켜줌.

In [0]:
sess=tf.InteractiveSession()

# **계산그래프**
python에서 효율적인 수치계산을 위해,tensorflow는 상호작용하는 연산을 그래프로 묶어 그 전체를 python바깥에서 실행시키는 방법을 사용.


# **소프트맥스 회귀(softmax regression)모델 구성**
다층 계층의 합성곱 신경망을 구성할 것임.

# **플레이스홀더(Placeholder)**
x,y_에 특정한 값을 부여한 것이 아님.

tensorflow가 계산을 실행할때, 값을 넣어줄 자리가 placeholder이다.

In [0]:
x = tf.placeholder(tf.float32, shape=[None, 784])  
#부동소수점실수,2차원 tensor. 784sms 28*28 크기를 가진 mnist이미지를 한줄로 펼친 크기
y_ = tf.placeholder(tf.float32, shape=[None, 10])   
#2차원 tensor,mnist이미지의 숫자 클래스(one-hot벡터로 나타냄.)

# **변수**
가중치(w),편향(b)
tensorflow는 이러한 변수를 다루기위해 Variable을 제공

Variable이란 tensorflow의 계산 그래프안에 있는 값(계산에의해 변경되거나, 계산에 사용될 수 있음.)


In [0]:
#Variable들은 세션이 시작되기전에 초기화되어야함.
W = tf.Variable(tf.zeros([784,10]))
#이미지벡터크기 784, 출력숫자클래스 10
b = tf.Variable(tf.zeros([10]))
#10차원 벡터

In [0]:
sess.run(tf.global_variables_initializer())
#Variable들 각각에 대해 미리 지정된 초기값을 넣어주는 역할

# **클래스 예측 및 비용 함수**

벡터화된 이미지인 x를 가중치 행렬인 W와 곱하고, 여기에 편향 b를 더한 뒤, 각각의 클래스에 대한 소프트맥스 함수의 결과를 계산

In [0]:
y=tf.nn.softmax(tf.matmul(x,W)+b)

In [0]:
cross_entropy=tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
#손실함수는 크로스-엔트로피함수 사용
#reduce_mean은 사용된 이미지들 각각에서 계산된 합의 평균을 구해주는 함수. y의 2번째 차원(reduction_indices=[1])의 원소들을 합함.

# **모델 훈련시키기**
tensorflow는 전체 계산 그래프의 정보가 입력되어있음. 

라이브러리가 자동으로 미분을 통해 각각 변수에대한 손실함수의 그레디언트를 계산

경사하강법을 이용하여 크로스 엔트로피를 최소화할 것임.

In [0]:
train_step=tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
#학습속도 0.5
#반환된 train_step은 실행되었을때, 경사하강법을 통해 각각의 매개변수 변화시킴. 
#모델을 훈련시키려면 반복해서 실행하면됨.

In [0]:
for i in range(1000):
  batch=mnist.train.next_batch(50)
  #50개의 다음 훈련데이터 샘플 추출
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})
  #train_step을 실행하여 텐서x와 y_에 훈련데이터 넣어줌.
  #feed_dict는 계산 그래프안에 어떤 텐서도 변경가능.


# **모델 평가**
tf.argmax로 한 차원에서 가장 큰 값의 인덱스를 반환해줌.

tf.argmax(y,1)은 모델이 입력받고 가장 그럴듯하다고 생각한 레이블이고,

tf.argmax(y_,1)은 실제레이블


In [0]:
correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
#불리언으로 이루어진 리스트 반환함. 

In [0]:
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
#부동소수점으로 형변환하면 예측결과정확도나옴. 

In [26]:
print(accuracy.eval(feed_dict={x:mnist.test.images,y_:mnist.test.labels}))

0.9161


# **다중 계층 합성곱 신경망**



In [0]:
#매번 모델 만들때매다 반복
def weight_variable(shape):
  initial=tf.truncated_normal(shape,stddev=0.1)
  #stddev는 표준편차를 줌.
  return tf.Variable(initial)

def bias_variable(shape):
  initial=tf.constant(0.1,shape=shape)
  #죽은 뉴런을 방지하기위해 편향을 작은양수(0.1)로 초기화함.
  return tf.Variable(initial)

# **합성곱(convolution), 풀링(pooling)**

합성곱:

풀링:

strides:

padding:



In [0]:
def conv2d(x,W):
  return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding="SAME")

def max_pool_2X2(x):
  return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")